In [2]:
# Imports
import pandas as pd
import numpy as np
import random

# Tests

In [3]:
headers = ["name","health","armor_class","initiative","initiative_bonus","team","group"]
turn_track = pd.DataFrame(columns=headers)
turn_track

,name,health,armor_class,initiative,initiative_bonus,team,group


In [4]:
character = {
    "name":"Mr.",
    "health":56,
    "armor_class":20,
    "initiative": 15,
    "initiative_bonus":2,
    "team":"Heroes",
    "group":"Alone"
}
turn_track.append(character,ignore_index=True)


,name,health,armor_class,initiative,initiative_bonus,team,group
0,Mr.,56,20,15,2,Heroes,Alone


In [5]:
headers = ["name","health","armor_class"]
filter_track = pd.DataFrame(columns=headers)
character = {
    "name":"Mrs.",
    "health":568,
    "armor_class":200
}
filter_track = filter_track.append(character,ignore_index=True)
# filter_track
filter_track[filter_track.columns[[True,False,True]]]

,name,armor_class
0,Mrs.,200


In [6]:
party1 = pd.read_csv(".\Party1.csv")
party1["group"] = np.nan

In [7]:
def read_import(path,import_groups=True):
    party = pd.read_csv(path)
    if import_groups and ('group' in list(party)):
        return party
    else:
        party["group"] = np.nan
        return party
party2=read_import(".\Party2.csv")
# party2

In [8]:
def initiative_based_group_assignment(groups:pd.DataFrame):
    df = groups.copy()
    df['total_initiative'] = df['initiative']+df['initiative_bonus']
    df.sort_values(by='total_initiative',ascending=False,inplace=True)
    df_count = {}
    team_order = df['team']
    group_placement = []
    lastTeam = None
    for team in team_order:
        if team != lastTeam :
            if team in df_count.keys() :
                df_count[team]+=1
            else :
                df_count[team]=1
            lastTeam = team
        group_placement.append(f"{team} {df_count[team]}")
    df['group']=group_placement
    return df.drop(columns="total_initiative")
party2_grouped = initiative_based_group_assignment(party2)
# party2_grouped

In [9]:
def roll(sided=20):
    return random.randint(1,sided)

def auto_initiative(groups:pd.DataFrame):
    df = groups.copy()
    df['initiative'] = df['initiative'].apply(lambda x: roll(20))
    return df
party2_random_init = auto_initiative(party2)
party2_random_init_grouped = initiative_based_group_assignment(party2_random_init)
# party2_random_init_grouped

In [10]:
def groups_list(turn_track:pd.DataFrame):
    return(turn_track['group'].unique())
# groups_list(party2_random_init_grouped)

In [11]:
def df_match_slice(df:pd.DataFrame,column,match):
    return df[df[column]==match]
group_to_split_df = df_match_slice(party2_random_init_grouped,'group','Stark 3')
# for member in group_to_split_df.index:
#     print(type(member))

In [12]:
party2_change_init = party2_random_init_grouped.copy()
character_selected = 'Eddard "Ned" Stark'
party2_change_init.loc[(party2_change_init['name'] == character_selected,'initiative')]=24
# party2_change_init

In [13]:
def df_set_match_slice(df:pd.DataFrame,column,match,new_data):
    df_slice = df[df[column] == match].copy()
    df_slice[column] = new_data
    df_copy = df.copy()
    df_copy.update(df_slice)
    return df_copy
# group_to_split_df = df_set_match_slice(party2_random_init_grouped,'group','Targaryen 1',['Test1','Test2'])
# group_to_split_df

In [14]:
type(group_to_split_df)

pandas.core.frame.DataFrame

In [38]:
def move_group(groups:pd.DataFrame,group_to_move,before_or_after,group_to_place):
    df = groups.copy()
    df.reset_index(drop=True,inplace=True)
    slice_group_to_move = df[df['group']==group_to_move].copy()
    df.drop(slice_group_to_move.index,inplace=True)
    df.reset_index(drop=True,inplace=True)
    slice_group_to_move.reset_index(drop=True,inplace=True)
    if before_or_after == "Before" :
        index_split_point = (df[df['group']==group_to_place].index[0]) #first index
    else :
        index_split_point = df[df['group']==group_to_place].index[-1]+1 #last index
    return pd.concat([df.iloc[:index_split_point],slice_group_to_move,df.iloc[index_split_point:]]).reset_index(drop=True)
move_test = party2_random_init_grouped.copy()
# move_test
# move_group(move_test,"Stark 3","Before","Stark 1")

Empty DataFrame
Columns: [name, group]
Index: []
                 name       group
0  Eddard "Ned" Stark  Winterfell
1       Catelyn Stark  Winterfell
2          Robb Stark  Winterfell
3         Sansa Stark  Winterfell
4          Arya Stark  Winterfell
                 name          group
0    Robert Baratheon  Kings Landing
1     Jaime Lannister  Kings Landing
2    Cersei Lannister  Kings Landing
3  Daenerys Targaryen   Nights Watch
4       Jorah Mormont   Nights Watch
5   Viserys Targaryen   Nights Watch
6            Jon Snow   Nights Watch


,name,health,armor_class,initiative,initiative_bonus,team,group,attributes
0,"Eddard ""Ned"" Stark",90,13,1,3,Stark,Winterfell,NaN
1,Catelyn Stark,90,16,3,3,Stark,Winterfell,NaN
2,Robb Stark,90,16,10,3,Stark,Winterfell,NaN
3,Sansa Stark,90,16,10,3,Stark,Winterfell,NaN
4,Arya Stark,90,16,10,3,Stark,Winterfell,NaN
5,Robert Baratheon,90,16,12,3,Stark,Kings Landing,NaN
6,Jaime Lannister,90,16,14,3,Lannister,Kings Landing,NaN
7,Cersei Lannister,90,16,4,3,Lannister,Kings Landing,NaN
8,Daenerys Targaryen,90,16,5,3,Targaryen,Nights Watch,NaN
9,Jorah Mormont,90,16,6,3,Targaryen,Nights Watch,NaN


In [18]:
def move_character(groups:pd.DataFrame,person_to_move,destination_group):
    df = groups.copy()
    df.reset_index(drop=True,inplace=True)
    slice_character_to_move = df[df['name']==person_to_move].copy()
    slice_character_to_move['group']=destination_group
    df.drop(slice_character_to_move.index,inplace=True)
    index_split_point = df[df['group']==destination_group].index[-1]+1 #last index
    return pd.concat([df.iloc[:index_split_point],slice_character_to_move,df.iloc[index_split_point:]]).reset_index(drop=True)
move_char_test = party2_random_init_grouped.copy()
# move_test
# move_character(move_test,"Arya Stark","Stark 2")

In [19]:
def move_character_to_new_group(groups:pd.DataFrame,person_to_move,new_group_name):
    df = groups.copy()
    old_group=df.loc[(df["name"]==person_to_move,"group")].values[0]
    df.loc[(df["name"]==person_to_move,"group")]=new_group_name
    if df[df['group']==old_group].empty :
        return df # If character was the only member of a group, no need to rearrange
    return move_group(df,new_group_name,"After",old_group)
move_char_new_group_test = party2_random_init_grouped.copy()

# move_character_to_new_group(move_char_new_group_test,"Cersei Lannister","Kinda Annoying")


In [20]:
def merge_groups(groups:pd.DataFrame,merge_group_1,merge_group_2,merged_name):
    df = groups.copy()
    df = move_group(df,merge_group_1,"After",merge_group_2)
    df['group'].replace([merge_group_1,merge_group_2],[merged_name,merged_name],inplace=True)
    return df

merge_test = party2_random_init_grouped.copy()
# merge_test
# merge_groups(merge_test,"Targaryen 1","Targaryen 2","Taco Tuesday")

In [43]:
groups_list(party2)[groups_list(party2)!='Winterfell']

array(['Kings Landing', 'Nights Watch'], dtype=object)

# Possible Classes

In [21]:
class Character:
    name : str
    health : int
    initiative : int
    team : str
    
    def __init__(self):
        self.name = ""
        self.health = 0
        self.initiative = 0
        self.team = None
    def __init__(self,name,health,initiative,team):
        self.name = name
        self.health = health
        self.initiative = initiative
        self.team = team

In [22]:
class GroupLoop:
    group : pd.DataFrame
    name : str

    def __init__(self):
        self.group = pd.DataFrame()
        self.name = "Unnamed Group"
    def __init__(self, list_of_characters,group_name):
        self.group = pd.DataFrame(list_of_characters)
        self.name = group_name
    def set_next_group(self, next_group):
        self.next_group = next_group